In [1]:
from __future__ import division, print_function

import os
import json
import numpy  as np
import pandas as pd

from glob                        import glob
from scipy                       import misc, ndimage
from scipy.ndimage.interpolation import zoom
from matplotlib                  import pyplot as plt

np.set_printoptions(precision = 4, linewidth = 100)

from keras.utils.data_utils     import get_file
from keras                      import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils     import get_file
from keras.models               import Sequential
from keras.layers.core          import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling       import GlobalAveragePooling2D

from keras.optimizers           import SGD, RMSprop, Adam
from keras.preprocessing        import image

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


We define the path to our data. The sample folder is used for prototyping purpose.

In [2]:
path = 'data/redux/'
# path = 'data/redux/sample/'

We first define a function that we will use as a preprocessing to get our data in the same format that the VGG network takes.

In [3]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3, 1, 1))
def vgg_preprocess(x):
    x = x - vgg_mean  # remove the mean value for each channel 
    return x[:, ::-1] # reverse the order of the colors to match the vgg network

Add a convolution block to the model. 

In [4]:
def add_conv_block(model, layers, filters):
    for _ in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Convolution2D(filters, 3, 3, activation = 'relu'))
    model.add(MaxPooling2D((2, 2), strides = (2, 2)))

Add a fully connected block to the model

In [5]:
def add_FC_block(model):
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.5))

We now build our model according to the VGG network architecture.

In [6]:
def create_vgg_model():
    model = Sequential()

    # We first substart the mean value from each channel in order to center our data on 0. 
    # We also change the order of the channels to match the vgg definition.
    model.add(Lambda(vgg_preprocess, input_shape = (3, 224, 224)))

    # Then we add the convolution blocks to the model.
    conv_parameters = [(2, 64), (2, 128), (3, 256), (3, 512), (3, 512)]
    for layers, filters in conv_parameters:
        add_conv_block(model, layers, filters)

    # We then flatten the data since the we will not use the spacial
    # information of the picture.
    model.add(Flatten())

    # We now add the fully connected blocks.
    add_FC_block(model)
    add_FC_block(model)

    # And to finish, we add the final softmax layer that output the predicted probability
    # for each category.
    model.add(Dense(1000, activation = 'softmax'))

    # Now that our model is created, we load the weights that have already been trained. 
    filename = 'vgg16.h5'
    f = get_file(filename, 'http://www.platform.ai/models/' + filename)
    model.load_weights(f)
    
    return model

We now define the finetune function which takes vgg model and replace the last layer by a new layer which will, after a retraining, predict the probability of being a cat or a dog.

In [7]:
def compile_model(model, lr = 0.001):
    model.compile(optimizer = Adam(lr = lr),
                  loss      = 'categorical_crossentropy',
                  metrics   = ['accuracy'])

def finetune_model(model, batches):
    model.pop()
    for layer in model.layers:
        layer.trainable = False # We do not want to change the weights of the core of the network.
    model.add(Dense(batches.nb_class, activation = 'softmax'))
    compile_model(model)

We now define the batch that we will use for the training of the network.

In [8]:
batch_size = 64

In [9]:
def get_batches(path, 
                gen        = image.ImageDataGenerator(), 
                shuffle    = True, 
                batch_size = 8, 
                class_mode = 'categorical'):
    parameters = {
        'directory'  : path,
        'target_size': (224, 224),
        'class_mode' : class_mode,
        'shuffle'    : shuffle,
        'batch_size' : batch_size
    }
    return gen.flow_from_directory(**parameters)

In [10]:
model = create_vgg_model()

In [11]:
batches     = get_batches(path + 'train', batch_size = batch_size)
val_batches = get_batches(path + 'valid', batch_size = 2 * batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


We now finetune the model so that it fits our needs.

In [12]:
finetune_model(model, batches)

In [13]:
fit_parameters = {
    'generator'         : batches,
    'samples_per_epoch' : batches.nb_sample,
    'nb_epoch'          : 1,
    'validation_data'   : val_batches,
    'nb_val_samples'    : val_batches.nb_sample
}
model.fit_generator(**fit_parameters)

Epoch 1/1
23000/23000 [==============================] - 570s - loss: 0.1218 - acc: 0.9682 - val_loss: 0.0821 - val_acc: 0.9770


Now that the model is finetuned, we can use it to make our predictions on the test set.

In [14]:
test_batches = get_batches(path + 'test', batch_size = batch_size * 2, shuffle = False, class_mode = None)
predictions  = model.predict_generator(test_batches, test_batches.nb_sample)  

Found 12500 images belonging to 1 classes.


Now that we have the predictions, we save them in a file in order to submit it to Kaggle.

In [15]:
ids    = np.array([int(name[8 : -4]) for name in test_batches.filenames])
is_dog = predictions[:, 1].clip(0.07, 0.93) # we clip the result to avoid being penalized by the log loss function.

In [16]:
submission_dict = {
    'id'    : ids,
    'label' : is_dog
}
submission_df = pd.DataFrame(submission_dict)

In [17]:
submission_filename = path + 'result/submission.csv'
submission_df.to_csv(submission_filename, index = False)

In [18]:
from IPython.display import FileLink
FileLink(submission_filename)

/home/ubuntu/courses/deeplearning1/nbs/data/redux/result/submission.csv